## ME447 Project 1
#### Amanda Schultz (amanda36) and Avanthi Obadage (obadage2)

### 2. 1/0 Knapsack Probelm 
#### Genetic Algorithm version

In [2]:
#initialize 
import numpy as np
A_data = np.load("A.npz") #dict format: n_items, capacity, item_values, item_weights
B_data = np.load("B.npz")

In [20]:
#Given
n_items = A_data["n_items"]
capacity = A_data["capacity"]
values = np.array(A_data["item_values"])
weights = np.array(A_data["item_weights"])


#### Representation 
##### Solutions are binary vectors; [1] is included and [0] is excluded

In [51]:
# Representation
pop_size = 50 # how big is the population?
population = np.random.randint(2, size=(pop_size, n_items)) #some inital population with random objects


#Fitness
def fitness(individual):
    """
    A function that calculates the weight and value of an individual solution (parent) from the population
    """
    total_weight = np.sum(individual * weights)
    total_value = np.sum(individual * values)
    if total_weight > capacity:
        return 0  # overweight 
    return total_value

#Deterministic Variation Selection
num_parents = pop_size // 2 #next generation is half of previous
def select_parents(population, num_parents):
    """
    Selects the top solutions (parents) with highest fitness.
    """
    population = np.array(population)  # ensure it's an array
    fitness_vals = np.array([fitness(ind) for ind in population])
    top_idx = np.argsort(fitness_vals)[-num_parents:]  # largest fitness
    top_idx = top_idx[::-1]  
    return population[top_idx]

#Recombination
def crossover(parents, pop_size):
    """
    Crosses best parents to make offspring (new solutions) with (hopefully) better fitness
    """
    n_items = parents.shape[1]
    offspring = []
    while len(offspring) < pop_size:
        for i in range(len(parents)//2):
            p1 = parents[i]
            p2 = parents[-(i+1)]
            point = np.random.randint(1, n_items)
            c1 = np.concatenate([p1[:point], p2[point:]])
            c2 = np.concatenate([p2[:point], p1[point:]])
            offspring.append(c1)
            if len(offspring) < pop_size:
                offspring.append(c2)
    return np.array(offspring[:pop_size])

#Mutation
mutation_prob = 0.05 #some probability that a bit will flip 
def mutate(offspring, mutation_prob):
    """
    Randomly decides when a bit will flip to escape any local minima
    """
    mutation = np.random.rand(*offspring.shape) < mutation_prob
    return np.logical_xor(offspring, mutation).astype(int)

#Environmental Selection
def environmental_selection(offspring, pop_size):
    """
    Selects the top offspring individuals to form the next generation.
    """
    offspring = np.array(offspring) 
    fitness_vals = np.array([fitness(ind) for ind in offspring])
    top_idx = np.argsort(fitness_vals)[-pop_size:]  # largest fitness
    return offspring[top_idx]


# Run simulation!
generations = 50

for gen in range(generations):
    parents = select_parents(population, num_parents)
    
    # 4. Crossover
    offspring = crossover(parents, pop_size * 2)  # create extra to select from
    
    # 5. Mutation
    offspring = mutate(offspring, mutation_prob)
    
    # 6. Environmental selection (elitist)
    population = environmental_selection(offspring, pop_size)

# Solutions
fitness_vals = np.array([fitness(ind) for ind in population])
best_idx = np.argmax(fitness_vals)
best_solution = population[best_idx]
best_value = fitness_vals[best_idx]

print("Best solution:", best_solution)
print("Total value:", best_value)


Best solution: [0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0
 0 0 1 0 0 0 0 0 1 0 1 1 0]
Total value: 0
